In [1]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn import preprocessing
import python_speech_features as mfcc
def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            deltas[i]=deltas[i]+j*((array[(i+j)%(rows)]-array[(rows+i-j)%(rows)]))
            j+=1
        deltas[i] = deltas[i]/10
    return deltas

def extract_features(audio,rate):
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft=2058,appendEnergy = True)    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

#Load the Gaussian gender Models
modelpath = "C:\\Users\\Prashu Gupta\\Downloads\\datasets\\development_set\\1newdata\\model\\"
gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
              in gmm_files]

# Read the test directory and get the list of test audio files 
sr,audio = read("C:\\Users\\Prashu Gupta\\Downloads\\datasets\\development_set\\1newdata\\d3.wav")
print(audio.shape)
vector   = extract_features(audio,sr)
log_likelihood = np.zeros(len(models)) 
    
for i in range(len(models)):
    gmm    = models[i]  #checking with each model one by one
    scores = np.array(gmm.score(vector))
    log_likelihood[i] = scores.sum()
    print(speakers[i],log_likelihood[i])
winner = np.argmax(log_likelihood)
print ("\tdetected as - ", speakers[winner])
time.sleep(1.0)

(80000,)


ValueError: The shape of X  is not compatible with self